## Get the data
csv headings: id, created_at, source, original_text, clean_text, favorite_count, retweet_count, hashtags, trend <br>
hashtags format: strings with comma separated hashtags

In [79]:
#File paths
US_tweets_file = './Data/USTweets.csv'
UK_tweets_file = './Data/UKTweets.csv'
glove_path = './glove.840B.300d/glove.840B.300d.txt'

In [80]:
import numpy as np

embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

C:\Users\user\AppData\Local\Temp\ipykernel_1940\876895944.py:7: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


Found 2195884 word vectors.


In [112]:
import csv

tweets = []
hashtags = [] #list of lists of hashtags e.g. hashtags[0] = ["hashtag1", "hashtag2"]
hashtags_strings = [] #list of hashtags string e.g. hashtags[0] = ["hashtag1, hashtag2"]


In [113]:
with open(UK_tweets_file) as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

In [114]:
with open(US_tweets_file) as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

## Initialize the tokenizers
Will use a specialized tokenizer for the hashtags because we need to encode all the hashtags. It also does not matter if the encoding of the tweets match the encoding of the hashtags.

In [115]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

In [116]:
hashtags_tokenizer = Tokenizer(oov_token="<OOV>")
hashtags_tokenizer.fit_on_texts(hashtags_strings)
hashtags_word_index = hashtags_tokenizer.word_index
hashtags_index_word = hashtags_tokenizer.index_word

In [117]:
print(f'There are {len(tweets)} tweets, ')
print(f'the tweets contain {len(tweets_tokenizer.word_index)} different words.')
print(f'There are {len(hashtags_tokenizer.word_index)} different hashtags')
print('Here are the tokenized hashtags')
print(hashtags_word_index)

There are 4045 tweets, 
the tweets contain 7706 different words.
There are 1949 different hashtags
Here are the tokenized hashtags
{'<OOV>': 1, 'nufc': 2, 'facup': 3, 'bhamun': 4, 'newtot': 5, 'mufc': 6, 'sundayvibes': 7, 'emergencyalert': 8, 'sundayfunday': 9, 'sundaymorning': 10, 'thfc': 11, 'spurs': 12, 'sunday': 13, 'newcastle': 14, 'tottenham': 15, 'weekendvibes': 16, 'barcaatleti': 17, 'coys': 18, 'barçaatleti': 19, 'levyout': 20, 'kkrvcsk': 21, 'premierleague': 22, 'freenbeckyfanboommnl': 23, 'stgeorgesday': 24, 'emiratesfacup': 25, 'godnightsunday': 26, 'sudan': 27, 'bhafc': 28, 'facupsemifinal': 29, 'godmorningsunday': 30, 'manutd': 31, 'timelessconcert': 32, 'newcastleunited': 33, 'aberdeen': 34, 'fpl': 35, 'emergencyalerts': 36, 'davisgarcia': 37, 'bbcfootball': 38, 'rangersfc': 39, 'srchafreen': 40, 'england': 41, 'rahane': 42, 'enicout': 43, 'milanlecce': 44, 'sempremilan': 45, 'brighton': 46, 'munbha': 47, 'catsoftwitter': 48, 'cats': 49, 'hwtl': 50, 'nufcfans': 51, 'epl'

In [118]:
num_tokens = len(tweets_word_index) + 1
embedding_dim = 300
hits = 0
misses = 0
print(num_tokens)

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tweets_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        if embedding_vector.shape == (300,):
            embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

7707


In [119]:
print(hits, misses)

6172 1534


## Create the sequences and pad them and one hot encode the hashtags
Will use a binary vector to encode the hashtags to the model can categorize the tweets. e.g. hashtags[0] = [tag1, tag2], and tag1 has encoding of 1 and tag2 has encoding 2, then the binary vector wil be [0 1 1 0 0 ... no_of_different_hashtags]

In [120]:
sequence_length = 10

from keras.utils import pad_sequences
tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
hashtags_sequences = hashtags_tokenizer.texts_to_sequences(hashtags)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

In [121]:
from keras.utils import to_categorical
import tensorflow as tf

no_of_different_hashtags = len(hashtags_word_index) + 1
no_of_hashtags = len(hashtags_sequences)

encoded_hashtags = np.zeros((no_of_hashtags, no_of_different_hashtags))

for i, hashtags_indices in enumerate(hashtags_sequences):
    encoded_hashtags[i][hashtags_indices] = 1


encoded_hashtags = np.array(encoded_hashtags)


## Build the model

In [122]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 100
dropout_value = 0.5
conv_filters = 64
conv_kernel_size = 5

In [123]:
from keras import initializers

no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(no_of_tweets_words, embedding_dimensions,
                               input_length=sequence_length,
                                 embeddings_initializer=initializers.Constant(embedding_matrix),
                                   trainable=False),
    # tf.keras.layers.Conv1D(conv_filters, conv_kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_value),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    # tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_different_hashtags, activation='softmax')
])

hashtag_recommender_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

hashtag_recommender_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 10, 300)           2312100   
                                                                 
 dropout_23 (Dropout)        (None, 10, 300)           0         
                                                                 
 bidirectional_15 (Bidirecti  (None, 10, 200)          320800    
 onal)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 10, 200)           0         
                                                                 
 bidirectional_16 (Bidirecti  (None, 200)              240800    
 onal)                                                           
                                                                 
 dropout_25 (Dropout)        (None, 200)              

## Train the model

In [124]:
epochs = 8
hashtag_recommender_model.fit(tweets_sequences_padded, encoded_hashtags, epochs=epochs)

Epoch 1/8
127/127 [==============================] - 7s 19ms/step - loss: 11.5943 - accuracy: 0.2282
Epoch 2/8
127/127 [==============================] - 2s 19ms/step - loss: 11.4114 - accuracy: 0.3070
Epoch 3/8
127/127 [==============================] - 2s 19ms/step - loss: 12.1408 - accuracy: 0.3310
Epoch 4/8
127/127 [==============================] - 2s 20ms/step - loss: 12.7295 - accuracy: 0.3454
Epoch 5/8
127/127 [==============================] - 3s 20ms/step - loss: 14.2865 - accuracy: 0.3461
Epoch 6/8
127/127 [==============================] - 2s 20ms/step - loss: 15.0405 - accuracy: 0.3600
Epoch 7/8
127/127 [==============================] - 2s 19ms/step - loss: 15.7543 - accuracy: 0.3649
Epoch 8/8
127/127 [==============================] - 3s 20ms/step - loss: 16.9177 - accuracy: 0.3644


## Get hashtags!!

In [125]:
def predict(tweet, tweet_tokenizer, hashtag_tokenizer, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    hashtag_index = np.argmax(prediction, axis=-1)[0]
    return hashtag_tokenizer.index_word[hashtag_index]


In [143]:
print(predict('oh my god', tweets_tokenizer, hashtags_tokenizer, 10, hashtag_recommender_model))

1/1 [==============================] - 0s 18ms/step
sundayvibes
